In [31]:
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator

# data_loading code from the official website
def load_cifar10_batch(cifar10_dataset_fold_path, batch_id):
    file = open(cifar10_dataset_fold_path + '/data_batch_' + str(batch_id), 'rb')
    batch = pickle.load(file, encoding='latin1')
    features = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
    labels = batch['labels']
    file.close()
    return features, labels

# load training data
cifar10_dataset_fold_path = '/Users/dayao/Desktop/data_mining/final_project/cifar-10-batches-py'
X_train, y_train = load_cifar10_batch(cifar10_dataset_fold_path, 1)
for i in range(2, 6):
    features, labels = load_cifar10_batch(cifar10_dataset_fold_path, i)
    X_train = np.concatenate((X_train, features))
    y_train = np.concatenate((y_train, labels))
    
# load test data
file = open(cifar10_dataset_fold_path + '/test_batch', 'rb')
batch = pickle.load(file, encoding='latin1')
X_test = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
y_test = batch['labels']
file.close()

# normalize the data
minmax = MinMaxScaler()
X_train_temp = X_train.reshape(X_train.shape[0], 32 * 32 * 3)
X_test_temp = X_test.reshape(X_test.shape[0], 32 * 32 * 3)
X_train = minmax.fit_transform(X_train_temp)
X_test = minmax.fit_transform(X_test_temp)
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

# one-hot encode the labels
num_classes = 10
nums_class = []
for j in range(1, num_classes + 1):
    nums_class.append(j)
lb = LabelBinarizer().fit(nums_class)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

# split training data and validation data
train_size = 0.75
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  train_size = train_size,
                                                  random_state = 7)

# build a simple CNN
batch_size = 64
epochs = 20
data_augmentation = False
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer= 'rmsprop',
              metrics=['accuracy'])

# fit the data to the model
if not data_augmentation:
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_val, y_val),
              shuffle=True)
else:
    datagen = ImageDataGenerator(rotation_range=90,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 vertical_flip=True)
    datagen.fit(X_train)
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        workers=4,
                        shuffle=True)

# score the trained model
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

C:\my_own_app\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\my_own_app\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\my_own_app\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 37500 samples, validate on 12500 samples
Epoch 1/20
37500/37500 [==============================] - 267s 7ms/step - loss: 1.5269 - acc: 0.3454 - val_loss: 1.1950 - val_acc: 0.4618
Epoch 2/20
37500/37500 [==============================] - 260s 7ms/step - loss: 1.1255 - acc: 0.4983 - val_loss: 0.9616 - val_acc: 0.5703
Epoch 3/20
37500/37500 [==============================] - 258s 7ms/step - loss: 0.9172 - acc: 0.5785 - val_loss: 0.8818 - val_acc: 0.5960
Epoch 4/20
37500/37500 [==============================] - 286s 8ms/step - loss: 0.7896 - acc: 0.6275 - val_loss: 0.7164 - val_acc: 0.6470
Epoch 5/20
37500/37500 [==============================] - 261s 7ms/step - loss: 0.7047 - acc: 0.6556 - val_loss: 0.7408 - val_acc: 0.6514
Epoch 6/20
37500/37500 [==============================] - 250s 7ms/step - loss: 0.6405 - acc: 0.6822 - val_loss: 0.6484 - val_acc: 0.6778
Epoch 7/20
37500/37500 [==============================] - 294s 8ms/step - loss: 0.5869 - acc: 0.6984 - val_loss: 0.6287 - 

In [3]:
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
%matplotlib notebook

# data_loading code from the official website
def load_cifar10_batch(cifar10_dataset_fold_path, batch_id):
    file = open(cifar10_dataset_fold_path + '/data_batch_' + str(batch_id), 'rb')
    batch = pickle.load(file, encoding='latin1')
    features = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
    labels = batch['labels']
    file.close()
    return features, labels

# load training data
cifar10_dataset_fold_path = '/Users/dayao/Desktop/data_mining/final_project/cifar-10-batches-py'
X_train, y_train = load_cifar10_batch(cifar10_dataset_fold_path, 1)
for i in range(2, 6):
    features, labels = load_cifar10_batch(cifar10_dataset_fold_path, i)
    X_train = np.concatenate((X_train, features))
    y_train = np.concatenate((y_train, labels))
    
# load test data
file = open(cifar10_dataset_fold_path + '/test_batch', 'rb')
batch = pickle.load(file, encoding='latin1')
X_test = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
y_test = batch['labels']
file.close()

# normalize the data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
minmax = MinMaxScaler()
X_train_temp = X_train.reshape(X_train.shape[0], 32 * 32 * 3)
X_test_temp = X_test.reshape(X_test.shape[0], 32 * 32 * 3)
X_train_HD = minmax.fit_transform(X_train_temp)
X_test_HD = minmax.fit_transform(X_test_temp)

# # determine the components number of PCA on training data
# pca_train = PCA().fit(X_train_HD)
# # plot the cumulative summation of the explained variance
# plt.figure()
# plt.plot(np.cumsum(pca_train.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)') # for each component
# plt.title('Cifar-10 Explained Variance of training data')
# plt.show() # we cawn see when the number of components is 675, we can preserve something around 99.28% of the total variance of the data

# # do the PCA, based on the figures, we choose 600 to be the number of components
# pca = PCA(n_components=300)
# X_train = pca.fit_transform(X_train_HD)
# X_test = pca.transform(X_test_HD)

# X_train = X_train.reshape(X_train.shape[0], 10, 10, 3)
# X_test = X_test.reshape(X_test.shape[0], 10, 10, 3)

# one-hot encode the labels
num_classes = 10
nums_class = []
for j in range(1, num_classes + 1):
    nums_class.append(j)
lb = LabelBinarizer().fit(nums_class)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

# split training data and validation data
train_size = 0.75
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  train_size = train_size,
                                                  random_state = 7)

# build a simple CNN
batch_size = 64
epochs = 20
data_augmentation = True
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay),
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer= 'rmsprop',
              metrics=['accuracy'])

model.summary()

# # fit the data to the model
# if not data_augmentation:
#     model.fit(X_train, y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(X_val, y_val),
#               shuffle=True)
# else:
#     datagen = ImageDataGenerator(rotation_range=45,
#                                  width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                  zoom_range=0.2,
#                                  vertical_flip=True,
#                                  fill_mode='nearest')
#     datagen.fit(X_train)
#     model.fit_generator(datagen.flow(X_train, y_train,
#                                      batch_size=batch_size),
#                         steps_per_epoch=len(datagen.flow(X_train, y_train,batch_size=batch_size)),
#                         epochs=epochs,
#                         validation_data=(X_val, y_val),
#                         workers=4,
#                         shuffle=True)

# # score the trained model
# scores = model.evaluate(X_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

C:\my_own_app\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_15 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_16 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 64)        18496     
__________